## Imports

In [ ]:
import jax
import pickle
import networkx
import pandas as pd
import numpy as onp
import jax.numpy as np
from dgl import data
from functools import partial
from tqdm import tqdm_notebook as tqdm
from scipy.spatial.distance import pdist
from sklearn.preprocessing import OneHotEncoder
from jax import random as r
from jax.experimental import optimizers
from jax.scipy.special import logsumexp
from jax.experimental.stax import Dropout
from jax.flatten_util import ravel_pytree

## Dataset

In [ ]:
cora = data.citation_graph.CoraDataset()
X = cora.features
y = cora.labels

G = cora.graph
A = networkx.to_numpy_matrix(G).A

Aself = A + np.eye(A.shape[0])
d = Aself.sum(axis=1)
W = np.dot(np.diag(1.0 / d), Aself)

train_idx = cora.train_mask.astype(onp.bool)
val_idx = cora.val_mask.astype(onp.bool)
test_idx = cora.test_mask.astype(onp.bool)

y_onehot = OneHotEncoder(sparse=False).fit_transform(y.reshape(-1, 1))

input_size = X.shape[1]
hidden_size = 64
output_size = onp.max(y) + 1

## Functions

In [ ]:
def cross_entropy(y, ypred):
    return - np.mean(np.sum(y * np.log(ypred), axis=1))


def accuracy(y, ypred):
    return np.mean(np.argmax(y, axis=1) == np.argmax(ypred, axis=1))


def init_params(key):
    _, subkey1 = r.split(key)
    theta_l1 = jax.nn.initializers.lecun_normal()(key=key,
                                                  shape=(input_size, hidden_size))
    theta_l2 = jax.nn.initializers.lecun_normal()(key=subkey1,
                                                  shape=(hidden_size, output_size))
    b_l1 = np.zeros((hidden_size))
    b_l2 = np.zeros((output_size))
    return [theta_l1, b_l1, theta_l2, b_l2]


def init_params_dist(key):
    _, *subkeys = r.split(key, num=N_agents+1)
    params = []
    for i in range(N_agents):
        params.append(init_params(subkeys[i]))
    return params


@partial(jax.jit, static_argnums=(3))
def dist_gcn(params, L, X, mode, key=r.PRNGKey(0)):
    _, dropout_fcn = Dropout(0.5, mode=mode)
    _, subkey = r.split(key)
    X = dropout_fcn(_, X, rng=key)

    H = np.zeros((X.shape[0], hidden_size))
    for i in range(N_agents):
        H = jax.ops.index_update(H, node_to_agent == i,
                                 np.dot(X[node_to_agent == i],
                                 params[i][0]) + params[i][1])

    H = np.dot(L, H)
    H = jax.nn.relu(H)
    H = dropout_fcn(_, H, rng=subkey)
 
    Y = np.zeros((X.shape[0], output_size))
    for i in range(N_agents):
        Y = jax.ops.index_update(Y, node_to_agent == i,
                                 np.dot(H[node_to_agent == i],
                                 params[i][2]) + params[i][3])
  
  return jax.nn.softmax(np.dot(L, Y), axis=1)


def loss(params, L, key):
    ypred = dist_gcn(params, L, X, 'train', key=key)
    losses = cross_entropy(y_onehot[train_idx, :], ypred[train_idx, :])
    return np.mean(losses), (ypred, losses)

loss_and_grad = jax.jit(jax.value_and_grad(loss, has_aux=True))


@jax.jit
def consensus_step(params, W):
    flatten = [ravel_pytree(p) for p in params]
    p_flat = np.vstack(f[0] for f in flatten)
    p_flat = np.dot(W, p_flat)
    return [flatten[i][1](p_flat[i]) for i in range(p_flat.shape[0])]


@jax.jit
def consensus_step_optim(params, W):
    states = [p[0] for p in params]
    flatten = [ravel_pytree(s) for s in states]
    p_flat = np.vstack(f[0] for f in flatten)
    p_flat = np.dot(W, p_flat)
    new_state = [flatten[i][1](p_flat[i]) for i in range(p_flat.shape[0])]
    return [(new_p,old_p[1],old_p[2])
            for new_p, old_p in jax.util.safe_zip(new_state,params)]
  
    
def communication_mask(N_agents, P, key):
    M_agents = np.zeros((N_agents,N_agents))
    indices = np.triu_indices(N_agents,k=1)
    agents_mask = r.bernoulli(p=P,
                              shape=(int(N_agents*(N_agents-1)/2),),
                              key=key)
    M_agents = jax.ops.index_update(M_agents,indices,agents_mask)
    M_agents += M_agents.T + np.eye(N_agents)
    return M_agents
    if check_connected(M_agents):
        return M_agents
    else:
        print("Disconnected graph, retrying...")
        _, subkey = r.split(key)
        return  communication_mask(N_agents,P, subkey)


def prune_connections(A, M_agents, node_to_agent):
    new_A = A
    if not M_agents.all():
        #remove connections between agents that cannot communicate
        for i in range(N_agents):
            I = (node_to_agent==i).astype('int32')
            for j in range(i+1,N_agents):
                J = (node_to_agent==j).astype('int32')
                if M_agents[i,j]==0:
                    IJ = np.outer(I,J)
                    new_A -= (A*IJ)
    return new_A


def compute_consensus_matrix(new_A, N_agents, node_to_agent):
    W_agents = np.zeros((N_agents,N_agents))
    if N_agents > 1:
    #count numbere of edges from one agent to the other
        for i in range(N_agents):
            I = (node_to_agent==i).astype('int16')
            for j in range(i,N_agents):
                J = (node_to_agent==j).astype('int16')
                IJ = np.outer(I,J)
                edges = (new_A * IJ).sum()
                W_agents = jax.ops.index_update(W_agents,(i,j),edges)
                W_agents = jax.ops.index_update(W_agents,(j,i),edges)

            W_agents /= W_agents.sum(axis=0) 
    else:
        W_agents = np.ones((1,1))

    return W_agents


def check_connected(M_agents):
    tot = M_agents
    tmp = M_agents
    for i in range(M_agents.shape[0]):
        tmp = np.matmul(tmp,M_agents)
        tot += tmp
    
    return (tot>0.).all()


def compute_consensus_matrix_optim(M_agents, gamma=0.1):
    from jax.numpy.linalg import eigh
    
    def loss(X, mask):
        return np.sum(np.abs(X[mask]))
    
    def manual_grad(X):
        return np.sign(X)
    
    def compute_eig(X, Pu):
        A = (X + X.T) / 2
        B = (np.eye(X.shape[0]) - Pu)
        return  eigh(B @ A @ B)
    
    def projection(X, gamma, Pu):
        w,v = compute_eig(X,Pu)
        lb = -1 + gamma 
        ub =  1 - gamma 
        w = jax.ops.index_update(w, w < lb, lb)
        w = jax.ops.index_update(w, w > ub, ub)
        M = v @ np.diag(w) @ v.T
        return Pu + M 

    mu_0 = 1e-2 # lr
    delta = 1e-2 # lr decay factor 0.5 < () < 1
    gamma = gamma # 1-gamma = max allowed eigavl 0 < () < 1
    mask = (~(M_agents).astype(bool)).astype(float)
    N_agents = M_agents.shape[0]

    Pu = np.ones((N_agents, N_agents)) / N_agents
    Xopt = np.array(Pu)
    for i in range(1, 500):
        mu = mu_0 / np.power(i,delta)
        g = manual_grad(Xopt) * mask
        Xopt = projection(Xopt - mu * g, gamma, Pu)
    return Xopt

## Initialization

In [ ]:
N_agents = 6
P = 0.75
key = r.PRNGKey(42)

M_agents = communication_mask(N_agents,P,key)

node_to_agent = r.randint(minval=0, maxval=N_agents, shape=(X.shape[0], ), key=key)

new_A = prune_connections(A, M_agents, node_to_agent)
new_Aself = new_A + onp.eye(new_A.shape[0])
new_d = new_Aself.sum(axis=1)
new_W = np.dot(onp.diag(1.0 / new_d), new_A + new_Aself)

agents_W = compute_consensus_matrix(new_A,N_agents,node_to_agent)

del new_A, new_Aself, new_d

params = init_params_dist(key)
agents_opts = []
agents_opts_states = []
for i in range(N_agents):
    agents_opts.append(optimizers.sgd(lr))
    agents_opts_states.append(agents_opts[i][0](params[i]))

## Training

In [ ]:
bar = tqdm(range(iters))
_, subkey = r.split(key)
best_acc = best_val_acc = best_epoch = 0
results =  {'tr_loss':[],'tr_acc':[],'te_acc':[],'best_acc':[],'best_epoch':[]}
for i in bar:
    subkey, subkey2 = r.split(subkey)
    (loss_i, ypred), grads = loss_and_grad(params, new_W, subkey2)

    if i>0:
        agents_opts_states = consensus_step(agents_opts_states, agents_W)

    for k in range(N_agents):
        agents_opts_states[k] = agents_opts[k][1](i, grads[k], agents_opts_states[k])
        params[k] = agents_opts[k][2](agents_opts_states[k])

    ypred = dist_gcn(params, new_W, X, 'test')

    train_acc_i = accuracy(y_onehot[train_idx, :], ypred[train_idx, :])
    val_acc_i = accuracy(y_onehot[val_idx, :], ypred[val_idx, :])
    test_acc_i = accuracy(y_onehot[test_idx, :], ypred[test_idx, :])

    if val_acc_i > best_val_acc:
        best_val_acc = val_acc_i
        best_acc = test_acc_i
        best_epoch = i

    results['tr_loss'].append(loss_i)
    results['tr_acc'].append(train_acc_i)
    results['te_acc'].append(test_acc_i)

results['best_acc'] = best_acc
results['best_epoch'] = best_epoch

## Results

In [ ]:
print(f"Best epoch:{results['best_epoch']}, Best Accuracy:{results['best_acc']}")